In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
import networkx as nx
import matplotlib.pyplot as plt
import beartype as bt

- Create Levi graph (hyper graph version) with Incidence structure
  - Incidence matrix, simple graph, edges sum to 2 --> this is always **sparse**
- ~~Bipartite structure: matrix with entity on one index and connections on another index~~
- Tidy graph: edgelist and edge metadata graph
- Make sure pandas categorical works with ints
  - in nb, ss example has index as ints, not categoricals
  - pandas_categoricals
- Levi pandas accessor:
  - accessor to series, checks for multiindex categorical
  - instead of pandas.sparse
  - can control results to work with networkx etc
- Grabble object: built out of data classes (?)
- 


In [2]:
# Create sample dataset

doc_ids = [str(x) for x in range(5)]
texts = [
    "the quick brown fox jumped over the lazy dog",
    "the fox jumped over the dog",
    "the dog jumped over ten dogs",
    "the one brown fox",
    "the ten brown dogs",
   ]

df = pd.DataFrame({'doc_id': doc_ids,
                    'text':texts,
                    })

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=['the'])
cv.fit(df['text'])
tokens = cv.get_feature_names_out()

In [4]:
# Create cooccurence matrix

results = cv.transform(df['text'])
coocc = pd.DataFrame(results.T.dot(results).toarray(), index=tokens, columns=tokens)
coocc

# features = cv.get_feature_names_out()
# df_res = pd.DataFrame(results.toarray())
# df_res

,brown,dog,dogs,fox,jumped,lazy,one,over,quick,ten
brown,3,1,1,2,1,1,1,1,1,1
dog,1,3,1,2,3,1,0,3,1,1
dogs,1,1,2,0,1,0,0,1,0,2
fox,2,2,0,3,2,1,1,2,1,0
jumped,1,3,1,2,3,1,0,3,1,1
lazy,1,1,0,1,1,1,0,1,1,0
one,1,0,0,1,0,0,1,0,0,0
over,1,3,1,2,3,1,0,3,1,1
quick,1,1,0,1,1,1,0,1,1,0
ten,1,1,2,0,1,0,0,1,0,2


In [5]:
# Create document term matrix (BIPARTITE STRUCTURE)

results = cv.transform(df['text'])
features = cv.get_feature_names_out()
doc_term = pd.DataFrame(results.toarray(), columns=features)
doc_term

,brown,dog,dogs,fox,jumped,lazy,one,over,quick,ten
0,1,1,0,1,1,1,0,1,1,0
1,0,1,0,1,1,0,0,1,0,0
2,0,1,1,0,1,0,0,1,0,1
3,1,0,0,1,0,0,1,0,0,0
4,1,0,1,0,0,0,0,0,0,1


In [ ]:
# def of_beartype(t: bt.Union[type, bt.Tuple[type, ...]]) -> Predicate[object]:
#     """ From `phantom.predicates.generic.of_type`

#     Create a new predicate that succeeds when its argument is bearable on ``t``.
#     """

#     def check(a: object) -> bool:
#         return is_bearable(a, t)

#     return check

In [ ]:
# TODO Example code to change for our use-case 

@pd.api.extensions.register_dataframe_accessor("levi")   #can also be df, series, or index
class LeviAccessor:
    def __init__(self, pandas_obj):
        self._validate(pandas_obj)
        self._obj = pandas_obj

    @staticmethod
    def _validate(obj):
        # TODO: use beartype
        # # verify there is a column latitude and a column longitude
        # if "latitude" not in obj.columns or "longitude" not in obj.columns:
        #     raise AttributeError("Must have 'latitude' and 'longitude'.")

    @property
    def center(self):
        # return the geographic center point of this DataFrame
        lat = self._obj.latitude
        lon = self._obj.longitude
        return (float(lon.mean()), float(lat.mean()))

    def plot(self):
        # plot this array's data on a map, e.g., using Cartopy
        pass